In [57]:
from __future__ import print_function
import datetime
import time
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import codecs
from model.crf import *
from model.lm_lstm_crf import *
import model.utils as utils
from model.evaluator import eval_wc
from model.predictor import predict_wc #NEW

import argparse
import json
import os
import sys
from tqdm import tqdm
import itertools
import functools
import random


In [2]:
def eprint(*args, **kwargs):
    print(*args, file=sys.stderr, **kwargs)

In [76]:
class args:
    train_file = ["data_bioner_5/BC2GM-IOBES/merge.tsv",
                  "data_bioner_5/BC4CHEMD-IOBES/merge.tsv",
                  "data_bioner_5/BC5CDR-IOBES/merge.tsv",
                  "data_bioner_5/NCBI-disease-IOBES/merge.tsv",
                  "data_bioner_5/JNLPBA-IOBES/merge.tsv"]
    
    dev_file = ["data_bioner_5/BC2GM-IOBES/devel.tsv",
                  "data_bioner_5/BC4CHEMD-IOBES/devel.tsv",
                  "data_bioner_5/BC5CDR-IOBES/devel.tsv",
                  "data_bioner_5/NCBI-disease-IOBES/devel.tsv",
                  "data_bioner_5/JNLPBA-IOBES/devel.tsv"]
    
    test_file = ["data_bioner_5/BC2GM-IOBES/test.tsv",
                  "data_bioner_5/BC4CHEMD-IOBES/test.tsv",
                  "data_bioner_5/BC5CDR-IOBES/test.tsv",
                  "data_bioner_5/NCBI-disease-IOBES/test.tsv",
                  "data_bioner_5/JNLPBA-IOBES/test.tsv"]
    
    rand_embedding = False
    emb_file = 'data_bioner_5/wikipedia-pubmed-and-PMC-w2v.txt'
    gpu = 1
    batch_size = 80
    unk = 'unk'
    char_hidden = 300
    word_hidden = 300
    drop_out = 0.5
    epoch = 100
    start_epoch = 0
    checkpoint = './checkpoint/model_{}'.format(epoch)
    caseless = True
    char_dim = 30
    word_dim = 200
    char_layers = 1
    word_layers = 1
    lr = 0.01
    lr_decay = 0.05
    fine_tune = False
    load_check_point = ''
    load_opt = False
    update = 'adam'
    momentum = 0.9
    clip_grad = 5.0
    small_crf = True
    mini_count = 5
    lambda0 = 1
    co_train = False
    patience = 30
    high_way = False
    highway_layers = 1
    eva_matrix = 'fa'
    least_iters = 50
    shrink_embedding = True
    output_annotation = False

In [7]:
print(args.lr)

0.01


In [22]:
if args.gpu >= 0:
    torch.cuda.set_device(args.gpu)

print('setting:')
print(args)

setting:
<class '__main__.args'>


- loading corpus

In [23]:
# load corpus
print('loading corpus')
file_num = len(args.train_file)
lines = []
dev_lines = []
test_lines = []
for i in range(file_num):
    with codecs.open(args.train_file[i], 'r', 'utf-8') as f:
        lines0 = f.readlines()
    lines.append(lines0)
for i in range(file_num):
    with codecs.open(args.dev_file[i], 'r', 'utf-8') as f:
        dev_lines0 = f.readlines()
    dev_lines.append(dev_lines0)
for i in range(file_num):
    with codecs.open(args.test_file[i], 'r', 'utf-8') as f:
        test_lines0 = f.readlines()
    test_lines.append(test_lines0)

dataset_loader = []
dev_dataset_loader = []
test_dataset_loader = []
f_map = dict()
l_map = dict()
char_count = dict()
train_features = []
dev_features = []
test_features = []
train_labels = []
dev_labels = []
test_labels = []
train_features_tot = []
test_word = []

loading corpus


In [24]:
# load corpus con't
for i in range(file_num):
    dev_features0, dev_labels0 = utils.read_corpus(dev_lines[i])
    test_features0, test_labels0 = utils.read_corpus(test_lines[i])

    dev_features.append(dev_features0)
    test_features.append(test_features0)
    dev_labels.append(dev_labels0)
    test_labels.append(test_labels0)

    if args.output_annotation: #NEW
        test_word0 = utils.read_features(test_lines[i])
        test_word.append(test_word0)

    if args.load_check_point:
        if os.path.isfile(args.load_check_point):
            print("loading checkpoint: '{}'".format(args.load_check_point))
            checkpoint_file = torch.load(args.load_check_point)
            args.start_epoch = checkpoint_file['epoch']
            f_map = checkpoint_file['f_map']
            l_map = checkpoint_file['l_map']
            c_map = checkpoint_file['c_map']
            in_doc_words = checkpoint_file['in_doc_words']
            train_features, train_labels = utils.read_corpus(lines[i])
        else:
            print("no checkpoint found at: '{}'".format(args.load_check_point))
    else:
        print('constructing coding table')
        train_features0, train_labels0, f_map, l_map, char_count = \
        utils.generate_corpus_char(lines[i], f_map, l_map, 
                                   char_count, c_thresholds=args.mini_count, if_shrink_w_feature=False)

    train_features.append(train_features0)
    train_labels.append(train_labels0)

    train_features_tot += train_features0

shrink_char_count = [k for (k, v) in iter(char_count.items()) if v >= args.mini_count]
char_map = {shrink_char_count[ind]: ind for ind in range(0, len(shrink_char_count))}

char_map['<u>'] = len(char_map)  # unk for char
char_map[' '] = len(char_map)  # concat for char
char_map['\n'] = len(char_map)  # eof for char

f_set = {v for v in f_map}
dt_f_set = f_set
f_map = utils.shrink_features(f_map, train_features_tot, args.mini_count)

l_set = set()

constructing coding table
constructing coding table
constructing coding table
constructing coding table
constructing coding table


In [77]:
print(args.fine_tune)
print(args.emb_file)
print("f_map")
print(len(dt_f_set))
print(args.caseless)
print(args.unk)
print(args.word_dim)
print(args.shrink_embedding)
print(args.patience)

False
data_bioner_5/wikipedia-pubmed-and-PMC-w2v.txt
f_map
107553
True
unk
200
True
30


In [25]:
for i in range(file_num):

    dt_f_set = functools.reduce(lambda x, y: x | y, map(lambda t: set(t), dev_features[i]), dt_f_set)
    dt_f_set = functools.reduce(lambda x, y: x | y, map(lambda t: set(t), test_features[i]), dt_f_set)

    l_set = functools.reduce(lambda x, y: x | y, map(lambda t: set(t), dev_labels[i]), l_set)
    l_set = functools.reduce(lambda x, y: x | y, map(lambda t: set(t), test_labels[i]), l_set)

if not args.rand_embedding:
    print("feature size: '{}'".format(len(f_map)))
    print('loading embedding')
    if args.fine_tune:  # which means does not do fine-tune
        f_map = {'<eof>': 0}
    f_map, embedding_tensor, in_doc_words = utils.load_embedding_wlm(args.emb_file, ' ', f_map, dt_f_set, args.caseless, args.unk, args.word_dim, shrink_to_corpus=args.shrink_embedding)
    print("embedding size: '{}'".format(len(f_map)))

for label in l_set:
    if label not in l_map:
        l_map[label] = len(l_map)

print('constructing dataset')

feature size: '26220'
loading embedding
embedding size: '68437'
constructing dataset


In [26]:
for i in range(file_num):
    # construct dataset
    print('File:{}\n'.format(i))
    tmp_st = time.time()
    dataset, forw_corp, back_corp = utils.construct_bucket_mean_vb_wc(train_features[i], train_labels[i], l_map, char_map, f_map, args.caseless)
    tmp_en = time.time()
    print('After train {} in {}s\n'.format(i, tmp_en-tmp_st))
    tmp_st = time.time()
    dev_dataset, forw_dev, back_dev = utils.construct_bucket_mean_vb_wc(dev_features[i], dev_labels[i], l_map, char_map, f_map, args.caseless)
    tmp_en = time.time()
    print('After dev {} in {}s\n'.format(i, tmp_en-tmp_st))
    tmp_st = time.time()
    test_dataset, forw_test, back_test = utils.construct_bucket_mean_vb_wc(test_features[i], test_labels[i], l_map, char_map, f_map, args.caseless)
    tmp_en = time.time()
    print('After test {} in {}s\n'.format(i, tmp_en-tmp_st))

    tmp_st = time.time()
    dataset_loader.append([torch.utils.data.DataLoader(tup, args.batch_size, shuffle=True, drop_last=False) for tup in dataset])
    tmp_en = time.time()
    print('After train_loader {} in {}s\n'.format(i, tmp_en - tmp_st))
    tmp_st = time.time()
    dev_dataset_loader.append([torch.utils.data.DataLoader(tup, 50, shuffle=False, drop_last=False) for tup in dev_dataset])
    tmp_en = time.time()
    print('After dev_loader {} in {}s\n'.format(i, tmp_en-tmp_st))
    tmp_st = time.time()
    test_dataset_loader.append([torch.utils.data.DataLoader(tup, 50, shuffle=False, drop_last=False) for tup in test_dataset])
    tmp_en = time.time()
    print('After test_loader {} in {}s\n'.format(i, tmp_en-tmp_st))

File:0

After train 0 in 159.03229427337646s

After dev 0 in 4.502795457839966s

After test 0 in 17.93623995780945s

After train_loader 0 in 0.0003192424774169922s

After dev_loader 0 in 7.295608520507812e-05s

After test_loader 0 in 7.05718994140625e-05s

File:1

After train 1 in 4312.685806274414s

After dev 1 in 685.8849020004272s

After test 1 in 512.8247437477112s

After train_loader 1 in 0.0001361370086669922s

After dev_loader 1 in 6.818771362304688e-05s

After test_loader 1 in 6.651878356933594e-05s

File:2

After train 2 in 55.187166690826416s

After dev 2 in 13.343783855438232s

After test 2 in 15.141234397888184s

After train_loader 2 in 0.00015115737915039062s

After dev_loader 2 in 6.890296936035156e-05s

After test_loader 2 in 6.604194641113281e-05s

File:3

After train 3 in 25.866610765457153s

After dev 3 in 0.6279408931732178s

After test 3 in 0.660031795501709s

After train_loader 3 in 0.0001804828643798828s

After dev_loader 3 in 6.532669067382812e-05s

After test_lo

In [27]:
print('building model')
ner_model = LM_LSTM_CRF(len(l_map), len(char_map), args.char_dim, args.char_hidden, args.char_layers, args.word_dim, args.word_hidden, args.word_layers, len(f_map), args.drop_out, file_num, large_CRF=args.small_crf, if_highway=args.high_way, in_doc_words=in_doc_words, highway_layers = args.highway_layers)

if args.load_check_point:
    ner_model.load_state_dict(checkpoint_file['state_dict'])
else:
    if not args.rand_embedding:
        ner_model.load_pretrained_word_embedding(embedding_tensor)
    ner_model.rand_init(init_word_embedding=args.rand_embedding)

if args.update == 'sgd':
    optimizer = optim.SGD(ner_model.parameters(), lr=args.lr, momentum=args.momentum)
elif args.update == 'adam':
    optimizer = optim.Adam(ner_model.parameters(), lr=args.lr)

if args.load_check_point and args.load_opt:
    optimizer.load_state_dict(checkpoint_file['optimizer'])

crit_lm = nn.CrossEntropyLoss()
crit_ner = CRFLoss_vb(len(l_map), l_map['<start>'], l_map['<pad>'])

if args.gpu >= 0:
    if_cuda = True
    print('device: ' + str(args.gpu))
    torch.cuda.set_device(args.gpu)
    crit_ner.cuda()
    crit_lm.cuda()
    ner_model.cuda()
    packer = CRFRepack_WC(len(l_map), True)
else:
    if_cuda = False
    packer = CRFRepack_WC(len(l_map), False)

tot_length = sum(map(lambda t: len(t), dataset_loader))

best_f1 = []
for i in range(file_num):
    best_f1.append(float('-inf'))

best_pre = []
for i in range(file_num):
    best_pre.append(float('-inf'))

best_rec = []
for i in range(file_num):
    best_rec.append(float('-inf'))

track_list = list()
start_time = time.time()
epoch_list = range(args.start_epoch, args.start_epoch + args.epoch)
patience_count = 0

evaluator = eval_wc(packer, l_map, args.eva_matrix)

predictor = predict_wc(if_cuda, f_map, char_map, l_map, f_map['<eof>'], char_map['\n'], l_map['<pad>'], l_map['<start>'], True, args.batch_size, args.caseless) #NEW


building model


/home/tingyang/.conda/envs/adv/lib/python3.10/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/home/tingyang/Github/Multi-BioNER/model/utils.py:797: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  nn.init.uniform(weight, -bias, bias)
/home/tingyang/Github/Multi-BioNER/model/utils.py:800: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  nn.init.uniform(weight, -bias, bias)
/home/tingyang/Github/Multi-BioNER/model/utils.py:786: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  nn.init.uniform(input_linear.weight, -bias, bias)


device: 1


In [52]:
optimizer = optim.Adam(ner_model.parameters(), lr=args.lr)
print(packer)
print(l_map)

{'O': 0, 'B-GENE': 1, 'I-GENE': 2, 'E-GENE': 3, 'S-GENE': 4, '<start>': 5, '<pad>': 6, 'S-Chemical': 7, 'B-Chemical': 8, 'E-Chemical': 9, 'I-Chemical': 10, 'B-Disease': 11, 'E-Disease': 12, 'I-Disease': 13, 'S-Disease': 14, 'B-DNA': 15, 'E-DNA': 16, 'B-protein': 17, 'E-protein': 18, 'S-protein': 19, 'I-protein': 20, 'B-cell_type': 21, 'I-cell_type': 22, 'E-cell_type': 23, 'I-DNA': 24, 'S-cell_type': 25, 'S-DNA': 26, 'S-cell_line': 27, 'B-cell_line': 28, 'E-cell_line': 29, 'I-cell_line': 30, 'B-RNA': 31, 'I-RNA': 32, 'E-RNA': 33, 'S-RNA': 34}


In [78]:
print(epoch_list)
print(args.eva_matrix)
if 'f' in args.eva_matrix:
    print("yeah")
    
print(file_no)
print(len(dataset_loader))
mmp = map(lambda t: len(t), dataset_loader)
epoch_list = range(args.start_epoch, args.start_epoch + args.epoch)
print(epoch_list)

range(0, 5)
fa
yeah
1
5
range(0, 100)


## Start Training in epochs

In [ ]:
import importlib

importlib.reload(utils)
importlib.reload(model.crf)
importlib.reload(model.evaluator)
from model.evaluator import eval_wc
#importlib.reload(eval_wc)
evaluator = eval_wc(packer, l_map, args.eva_matrix)

start_time = time.time()
for epoch_idx, args.start_epoch in enumerate(epoch_list):

    sample_num = 1

    epoch_loss = 0
    ner_model.train()

    for sample_id in tqdm( range(sample_num) , mininterval=2,
            desc=' - Tot it %d (epoch %d)' % (tot_length, args.start_epoch), leave=False, file=sys.stdout):
        
        file_no = random.randint(0, file_num-1)            
        cur_dataset = dataset_loader[file_no]

        for f_f, f_p, b_f, b_p, w_f, tg_v, mask_v, len_v in itertools.chain.from_iterable(cur_dataset):

            f_f, f_p, b_f, b_p, w_f, tg_v, mask_v = packer.repack_vb(f_f, f_p, b_f, b_p, w_f, tg_v, mask_v, len_v)

            ner_model.zero_grad()
            scores = ner_model(f_f, f_p, b_f, b_p, w_f, file_no)
            loss = crit_ner(scores, tg_v, mask_v)

            epoch_loss += utils.to_scalar(loss)
            if args.co_train:
                cf_p = f_p[0:-1, :].contiguous()
                cb_p = b_p[1:, :].contiguous()
                cf_y = w_f[1:, :].contiguous()
                cb_y = w_f[0:-1, :].contiguous()
                cfs, _ = ner_model.word_pre_train_forward(f_f, cf_p)
                loss = loss + args.lambda0 * crit_lm(cfs, cf_y.view(-1))
                cbs, _ = ner_model.word_pre_train_backward(b_f, cb_p)
                loss = loss + args.lambda0 * crit_lm(cbs, cb_y.view(-1))
            loss.backward()
            nn.utils.clip_grad_norm(ner_model.parameters(), args.clip_grad)
            optimizer.step()

    epoch_loss /= tot_length

    # update lr
    utils.adjust_learning_rate(optimizer, args.lr / (1 + (args.start_epoch + 1) * args.lr_decay))

    # eval & save check_point
    if 'f' in args.eva_matrix:
        dev_f1, dev_pre, dev_rec, dev_acc = evaluator.calc_score(ner_model, dev_dataset_loader[file_no], file_no)

        if dev_f1 > best_f1[file_no]:
            patience_count = 0
            best_f1[file_no] = dev_f1
            best_pre[file_no] = dev_pre
            best_rec[file_no] = dev_rec

            test_f1, test_pre, test_rec, test_acc = evaluator.calc_score(ner_model, test_dataset_loader[file_no], file_no)

            track_list.append(
                {'loss': epoch_loss, 'dev_f1': dev_f1, 'dev_acc': dev_acc, 'test_f1': test_f1,
                 'test_acc': test_acc})

            print(
                '(loss: %.4f, epoch: %d, dataset: %d, dev F1 = %.4f, dev pre = %.4f, dev rec = %.4f, F1 on test = %.4f, pre on test= %.4f, rec on test= %.4f), saving...' %
                (epoch_loss,
                 args.start_epoch,
                 file_no,
                 dev_f1,
                 dev_pre,
                 dev_rec,
                 test_f1,
                 test_pre,
                 test_rec))

            if args.output_annotation: #NEW
                print('annotating')
                with open('output'+str(file_no)+'.txt', 'w') as fout:
                    predictor.output_batch(ner_model, test_word[file_no], fout, file_no)

            try:
                utils.save_checkpoint({
                    'epoch': args.start_epoch,
                    'state_dict': ner_model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'f_map': f_map,
                    'l_map': l_map,
                    'c_map': char_map,
                    'in_doc_words': in_doc_words
                }, {'track_list': track_list
                    }, args.checkpoint + 'cwlm_lstm_crf')
            except Exception as inst:
                print(inst)

        else:
            patience_count += 1
            print('(loss: %.4f, epoch: %d, dataset: %d, dev F1 = %.4f, dev pre = %.4f, dev rec = %.4f)' %
                  (epoch_loss,
                   args.start_epoch,
                   file_no,
                   dev_f1,
                   dev_pre,
                   dev_rec))
            track_list.append({'loss': epoch_loss, 'dev_f1': dev_f1, 'dev_acc': dev_acc})

    print('epoch: ' + str(args.start_epoch) + '\t in ' + str(args.epoch) + ' take: ' + str(
        time.time() - start_time) + ' s')

    if patience_count >= args.patience and args.start_epoch >= args.least_iters:
        break

 - Tot it 20 (epoch 0):   0%|                                                                           | 0/1 [00:00<?, ?it/s]

/tmp/ipykernel_14907/2792602752.py:43: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  nn.utils.clip_grad_norm(ner_model.parameters(), args.clip_grad)


(loss: 5.0275, epoch: 0, dataset: 3, dev F1 = 0.8773, dev pre = 0.8870, dev rec = 0.8679, F1 on test = 0.7945, pre on test= 0.8485, rec on test= 0.7469), saving...
epoch: 0	 in 100 take: 6.446596622467041 s
(loss: 9.1468, epoch: 1, dataset: 2, dev F1 = 0.9388, dev pre = 0.9464, dev rec = 0.9314)                                     
epoch: 1	 in 100 take: 16.524696588516235 s
(loss: 17.0435, epoch: 2, dataset: 0, dev F1 = 0.8089, dev pre = 0.8109, dev rec = 0.8069, F1 on test = 0.7183, pre on test= 0.7337, rec on test= 0.7036), saving...
epoch: 2	 in 100 take: 35.71427869796753 s
